In [1]:
import os
from pathlib import Path
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType
from tqdm import tqdm
import pandas as pd

In [2]:
spark = SparkSession.builder.appName("SEV").getOrCreate()

23/06/30 03:35:32 WARN Utils: Your hostname, sevmon resolves to a loopback address: 127.0.1.1; using 143.106.60.101 instead (on interface ens3)
23/06/30 03:35:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/30 03:35:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
schema = StructType([
    StructField("data", StringType(), True),
    StructField("mensagem", StringType(), True),
    StructField("uid", StringType(), True),
    StructField("app", StringType(), True),
    StructField("motivo", StringType(), True),
    StructField("code", StringType(), True)
])
columns = ["nome", "uf", "cid", "zid", "uid", "sid", "data", "mensagem", "app", "motivo", "code", "file_path"]

In [4]:
cdd = pd.read_csv("/home/sevmon/scripts/municipios_brasileiros_tse.csv")

In [7]:
for subdir, dirs, files in os.walk("/home/sevmon/2t"):
    path = Path(subdir)
    if path.name == "2t_sp":
        print(f"Saving for state {path.name}")
        for file_name in tqdm(files):
            if file_name.startswith("o00407") and file_name.endswith(".dat"):
                file_path = os.path.join(subdir, file_name)  # Get the full file path
                
                cid = int(file_path[29:34])
                zid = int(file_path[34:38])
                sid = int(file_path[38:42])
                nome = cdd.loc[cdd.codigo_tse == cid].nome_municipio.iloc[0]

                df = spark.read \
                    .option("sep", "\t") \
                    .option("header", False) \
                    .option("inferSchema", False) \
                    .schema(schema) \
                    .csv(file_path)

                # Add the name of the folder to 'uf' column
                df = df.withColumn("uf", lit(str(path.name).split('_')[1].upper()))
                
                # Add the path of the file to 'file_path' column
                df = df.withColumn("file_path", lit(file_path))

                df = df.withColumn("cid", lit(cid))

                df = df.withColumn("zid", lit(zid))

                df = df.withColumn("sid", lit(sid))

                df = df.withColumn("nome", lit(nome))

                df = df.select(*columns)

                df.write.parquet(f"../data/{file_name[:-4]}")

Saving for state 2t_sp


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/home/sevmon/miniconda3/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/sevmon/miniconda3/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/sevmon/miniconda3/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
  0%|          | 0/107286 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
cdd.loc[cdd.codigo_tse == int(file_path[29:34])].nome_municipio.iloc[0]

'RIO BRANCO'

In [ ]:
cdd

,codigo_tse,uf,nome_municipio,capital,codigo_ibge
0,1120,AC,ACRELÂNDIA,0,1200013
1,1570,AC,ASSIS BRASIL,0,1200054
2,1058,AC,BRASILÉIA,0,1200104
3,1007,AC,BUJARI,0,1200138
4,1015,AC,CAPIXABA,0,1200179
...,...,...,...,...,...
5565,96199,TO,TOCANTÍNIA,0,1721109
5566,73458,TO,TUPIRAMA,0,1721257
5567,73237,TO,TUPIRATINS,0,1721307
5568,96652,TO,WANDERLÂNDIA,0,1722081


'o00407-6361402270410'